In [1]:
from mxnet.gluon import loss as gloss, nn, utils as gutils
from mxnet import autograd, nd, sym, gluon, init
import mxnet as mx
# import d2lzh as d2l
# import random
import subprocess
import time
import os 

# 08. 计算性能
在深度学习中，数据集通常很大而且模型计算往往很复杂。因此，我们十分关注计算性能。本章将重点介绍影响计算性能的重要因子：命令式编程、符号式编程、异步计算、自动并行计算和多GPU计算。

## 8.1 命令式和符号式混合编程
本书到目前为止一直都在使用命令式编程，它使用编程语句改变程序状态。考虑下面这段简单的命令式程序：

In [2]:
def add(a, b): 
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b) 
    f = add(c, d) 
    g = add(e, f)
    return g

fancy_func(1, 2, 3, 4)

10

和我们预期的一样，在运行语句`e = add(a, b)`时，Python会做加法运算并将结果存储在变量`e`中，从而令程序的状态发生改变。类似地，后面的2条语句会依次做加法运算并存储变量。

虽然使用命令式编程很方便，但它的运行可能很慢。一方面，即使`fancy_func`函数中的add是被重复调用的函数，Python也会逐一执行这3条函数调用语句。另一方面，我们需要保存变量`e`和`f`的值直到fancy_func中所有语句执行结束。这是因为在执行`e = add(a, b)`和`f = add(c, d)`这2条语句之后我们并不知道变量`e`和`f`是否会被程序的其他部分使用。

与命令式编程不同，符号式编程通常在计算流程完全定义好后才被执行。多个深度学习框架，如Theano和TensorFlow，都使用了符号式编程。通常，符号式编程的程序需要下面3个步骤：
+ 定义计算流程
+ 把计算流程编译成可执行的程序
+ 给定输入，调用编译好的程序执行

下面我们用符号式编程重新实现本节开头给出的命令式编程代码。

In [3]:
def add_str():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_str():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_str():
    return add_str() + fancy_func_str() + '''
print(fancy_func(1, 2, 3, 4))
'''

prog = evoke_str()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


以上定义的3个函数都仅以字符串的形式返回计算流程。最后，我们通过`compile`函数编译完整的计算流程并运行。由于在编译时系统能够完整地获取整个程序，因此有更多空间优化计算。例如，编译的时候可以将程序改写成`print((1 + 2) + (3 + 4))`，甚至直接改写成`print(10)`。这样不仅减少了函数调用，还节省了内存。

对比这两种编程方式，我们可以看到以下两点：
+ 命令式编程更方便、命令式编程更容易调试。这是因为我们可以很方便地获取并打印所有的中间变量值
+ 符号式编程更高效并更容易移植。一方面，在编译的时候系统容易做更多优化；另一方面，符号式编程可以将程序变成一个与Python无关的格式，从而可以使程序在非Python环境下运行，以避开Python解释器的性能问题

### 8.1.1 混合式编程取两者之长
大部分深度学习框架在命令式编程和符号式编程之间二选一。例如，Theano和受其启发的后来者TensorFlow使用了符号式编程，Chainer和它的追随者PyTorch使用了命令式编程。开发人员在设计Gluon时思考了这个问题：有没有可能既得到命令式编程的好处，又享受符号式编程的优势？开发者们认为，用户应该用纯命令式编程进行开发和调试；当需要产品级别的计算性能和部署时，用户可以将大部分命令式程序转换成符号式程序来运行。Gluon通过提供混合式编程的方式做到了这一点。

在混合式编程中，我们可以通过使用`HybridBlock`类或者`HybridSequential`类构建模型。默认情况下，它们和`Block`类或者`Sequential`类一样依据命令式编程的方式执行。当我们调用`hybridize`函数后，`Gluon`会转换成依据符号式编程的方式执行。事实上，绝大多数模型都可以接受这样的混合式编程的执行方式。

### 8.1.2 使用HybridSequential类构造模型
为了使用混合式编程，下面我们将`Sequential`类替换成`HybridSequential`类。

In [4]:
def get_net():
    net = nn.HybridSequential() # 这⾥创建HybridSequential实例 
    net.add(
        nn.Dense(256, activation='relu'),
        nn.Dense(128, activation='relu'),
        nn.Dense(2)) 
    net.initialize() 
    return net

x = nd.random.normal(shape=(1, 512)) 
net = get_net() 
net(x)


[[0.08827586 0.0050518 ]]
<NDArray 1x2 @cpu(0)>

我们可以通过调⽤`hybridize`函数来编译和优化`HybridSequential`实例中串联的层的计算。 模型的计算结果不变。

In [5]:
net.hybridize() 
net(x)


[[0.08827586 0.0050518 ]]
<NDArray 1x2 @cpu(0)>

需要注意的是，只有继承`HybridBlock`类的层才会被优化计算。例如，`HybridSequential`类和`Gluon`提供的`Dense`类都是`HybridBlock`类的⼦类，它们都会被优化计算。

##### 计算性能
下面通过比较调用`hybridize`函数前后的计算时间来展示符号式编程的性能提升。

In [6]:
def benchmark(net, x):
    start = time.time() 
    for i in range(1000):
        _ = net(x) 
    nd.waitall() # 等待所有计算完成⽅便计时 
    return time.time() - start

net = get_net() 
print('before hybridizing: %.4f sec' % (benchmark(net, x))) 
net.hybridize() 
print('after hybridizing: %.4f sec' % (benchmark(net, x)))

before hybridizing: 0.4943 sec
after hybridizing: 0.2053 sec


由上述结果可⻅，在⼀个`HybridSequential`实例调⽤`hybridize`函数后，它可以通过符号式编程提升计算性能。

##### 获取符号式程序
在模型`net`根据输入计算模型输出后，我们就可以通过`export`函数将符号式程序和模型参数保存到硬盘。

```python
net.export('my_mlp')
```

此时生成的`.json`和`.params`文件分别为符号式程序和模型参数。它们可以被`Python`或`MXNet`支持的其他前端语言读取。这样，我们就可以很方便地使用其他前端语言或在其他设备上部署训练好的模型。同时，由于部署时使用的是符号式程序，计算性能往往比命令式程序的性能更好。

在`MXNet`中，符号式程序指的是基于`Symbol`类型的程序。我们知道，当给`net`提供`NDArray`类型的输入`x`后，`net(x)`会根据`x`直接计算模型输出并返回结果。对于调用过`hybridize`函数后的模型，我们还可以给它输入一个`Symbol`类型的变量，`net(x)`会返回`Symbol`类型的结果。

In [7]:
x = sym.var('data') 
net(x)

<Symbol dense5_fwd>

### 8.1.3 使⽤HybridBlock类构造模型
和`Sequential`类与`Block`类之间的关系⼀样，`HybridSequential`类是`HybridBlock`类的⼦类。与`Block`实例需要实现`forward`函数不太⼀样的是，对于`HybridBlock`实例，我们需要实现`hybrid_forward`函数。

前⾯我们展⽰了调⽤`hybridize`函数后的模型可以获得更好的计算性能和可移植性。此外，调⽤`hybridize`函数后的模型会影响灵活性。

In [8]:
class HybridNet(nn.HybridBlock):
    def __init__(self, **kwargs):
        super(HybridNet, self).__init__(**kwargs) 
        self.hidden = nn.Dense(10) 
        self.output = nn.Dense(2)

    def hybrid_forward(self, F, x):
        print('F: ', F) 
        print('x: ', x) 
        x = F.relu(self.hidden(x)) 
        print('hidden: ', x)
        return self.output(x)

在继承`HybridBlock`类时，我们需要在`hybrid_forward`函数中添加额外的输⼊`F`。我们知道，`MXNet`既有基于命令式编程的`NDArray`类，⼜有基于符号式编程的`Symbol`类。由于这两个类的函数基本⼀致，`MXNet`会根据输⼊来决定`F`使⽤`NDArray`或`Symbol`。

下⾯创建了⼀个`HybridBlock`实例。可以看到在默认情况下`F`使⽤`NDArray`。而且，我们打印出了输⼊`x`和使⽤`ReLU`激活函数的隐藏层的输出。

In [9]:
net = HybridNet() 
net.initialize()
x = nd.random.normal(shape=(1, 4))
net(x)

F:  <module 'mxnet.ndarray' from '/home/alex/3rd/py-venv/lib/python3.6/site-packages/mxnet/ndarray/__init__.py'>
x:  
[[-0.12225834  0.5429998  -0.9469352   0.59643304]]
<NDArray 1x4 @cpu(0)>
hidden:  
[[0.11134676 0.04770704 0.05341475 0.         0.08091211 0.
  0.         0.04143535 0.         0.        ]]
<NDArray 1x10 @cpu(0)>



[[0.00370749 0.00134991]]
<NDArray 1x2 @cpu(0)>

再运⾏⼀次前向计算会得到同样的结果。

In [10]:
net(x)

F:  <module 'mxnet.ndarray' from '/home/alex/3rd/py-venv/lib/python3.6/site-packages/mxnet/ndarray/__init__.py'>
x:  
[[-0.12225834  0.5429998  -0.9469352   0.59643304]]
<NDArray 1x4 @cpu(0)>
hidden:  
[[0.11134676 0.04770704 0.05341475 0.         0.08091211 0.
  0.         0.04143535 0.         0.        ]]
<NDArray 1x10 @cpu(0)>



[[0.00370749 0.00134991]]
<NDArray 1x2 @cpu(0)>

接下来看看调⽤`hybridize`函数后会发⽣什么。

In [11]:
net.hybridize() 
net(x)

F:  <module 'mxnet.symbol' from '/home/alex/3rd/py-venv/lib/python3.6/site-packages/mxnet/symbol/__init__.py'>
x:  <Symbol data>
hidden:  <Symbol hybridnet0_relu0>



[[0.00370749 0.00134991]]
<NDArray 1x2 @cpu(0)>

可以看到，`F`变成了`Symbol`。而且，虽然输⼊数据还是`NDArray`，但在`hybrid_forward`函数⾥，相同输⼊和中间输出全部变成了`Symbol`类型。

再运⾏⼀次前向计算看看。

In [12]:
net(x)


[[0.00370749 0.00134991]]
<NDArray 1x2 @cpu(0)>

可以看到`hybrid_forward`函数里定义的3条打印语句都没有打印任何东西。这是因为上一次在调用`hybridize`函数后运行`net(x)`的时候，符号式程序已经得到。之后再运行`net(x)`的时候`MXNet`将不再访问Python代码，而是直接在C++后端执行符号式程序。这也是调用`hybridize`函数后模型计算性能会提升的一个原因。但它可能的问题在于，我们损失了写程序的灵活性。在上面这个例子中，如果我们希望使用那3条打印语句调试代码，执行符号式程序时会跳过它们无法打印。此外，对于少数像`asnumpy`这样的`Symbol`所不支持的函数，以及像`a += b`和`a[:] = a + b`（需改写为`a = a + b`）这样的`原地`(in-place)操作，我们无法在`hybrid_forward`函数中使用并在调用`hybridize`函数后进行前向计算。

## 8.2 异步计算
`MXNet`使⽤异步计算来提升计算性能。理解它的⼯作原理既有助于开发更⾼效的程序，⼜有助于在内存资源有限的情况下主动降低计算性能从而减小内存开销。

### 8.2.1 MXNet中的异步计算
广义上讲，`MXNet`包括用户直接用来交互的前端和系统用来执行计算的后端。例如，用户可以使用不同的前端语言编写MXNet程序，如Python、R、Scala和C++。无论使用何种前端编程语言，`MXNet`程序的执行主要都发生在C++实现的后端。换句话说，用户写好的前端MXNet程序会传给后端执行计算。后端有自己的线程在队列中不断收集任务并执行它们。

`MXNet`通过前端线程和后端线程的交互实现异步计算，即前端线程无须等待当前指令从后端线程返回结果就继续执行后面的指令。

假设Python前端线程调用以下4条指令。

In [13]:
a = nd.ones((1, 2)) 
b = nd.ones((1, 2))
c = a * b + 2
c


[[3. 3.]]
<NDArray 1x2 @cpu(0)>

在异步计算中，Python前端线程执行前3条语句的时候，仅仅是把任务放进后端的队列里就返回了。当最后一条语句需要打印计算结果时，Python前端线程会等待C++后端线程把变量`c`的结果计算完。此设计的一个好处是，这里的Python前端线程不需要做实际计算。因此，无论Python的性能如何，它对整个程序性能的影响很小。只要C++后端足够高效，那么不管前端编程语言性能如何，MXNet都可以提供一致的高性能。

为了演示异步计算的性能，我们先实现一个简单的计时类。

In [14]:
class Benchmark(): # 本类已保存在d2lzh包中⽅便以后使⽤
    def __init__(self, prefix=None):
        self.prefix = prefix + ' ' if prefix else ''

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, *args):
        print('%stime: %.4f sec' % (self.prefix, time.time() - self.start))

下面的例子通过计时来展示异步计算的效果。可以看到，当`y = nd.dot(x, x).sum()`返回的时候并没有等待变量`y`真正被计算完。只有当`print`函数需要打印变量`y`时才必须等待它计算完。

In [15]:
with Benchmark('Workloads are queued.'):
    x = nd.random.uniform(shape=(2000, 2000))
    y = nd.dot(x, x).sum()

with Benchmark('Workloads are finished.'):
    print('sum =', y)

Workloads are queued. time: 0.0014 sec
sum = 
[1.9998138e+09]
<NDArray 1 @cpu(0)>
Workloads are finished. time: 0.3517 sec


的确，除非我们需要打印或者保存计算结果，否则我们基本无须关心目前结果在内存中是否已经计算好了。只要数据是保存在`NDArray`里并使用`MXNet`提供的运算符，`MXNet`将默认使用异步计算来获取高计算性能。


### 8.2.2 用同步函数让前端等待计算结果
除了`print`函数外，我们可以使用`wait_to_read`函数让前端等待某个的`NDArray`的计算结果完成，再执行前端中后面的语句。或者，我们可以用`waitall`函数令前端等待前面所有计算结果完成。后者是性能测试中常用的方法。

In [16]:
with Benchmark():
    x = nd.random.uniform(shape=(2000, 2000))
    y = nd.dot(x, x).sum()
    y.wait_to_read()

with Benchmark('Workloads are finished.'):
    print('sum =', y)

time: 0.3558 sec
sum = 
[1.9995514e+09]
<NDArray 1 @cpu(0)>
Workloads are finished. time: 0.0011 sec


下⾯是使⽤`waitall`函数的例⼦。输出⽤时包含了变量`y`和变量`z`的计算时间。

In [17]:
with Benchmark():
    y = nd.dot(x, x) 
    z = nd.dot(x, x) 
    nd.waitall()

time: 0.6061 sec


此外，任何将`NDArray`转换成其他不⽀持异步计算的数据结构的操作都会让前端等待计算结果。 例如，当我们调⽤`asnumpy`函数和`asscalar`函数时：

In [18]:
with Benchmark():
    y = nd.dot(x, x) 
    y.asnumpy()

time: 0.3111 sec


In [19]:
with Benchmark():
    y = nd.dot(x, x) 
    y.norm().asscalar()

time: 0.3302 sec


上面介绍的`wait_to_read`函数、`waitall`函数、`asnumpy`函数、`asscalar`函数和`print`函数会触发让前端等待后端计算结果的行为。这类函数通常称为**同步函数**。

### 8.2.3 使⽤异步计算提升计算性能
在下面的例子中，我们用`for`循环不断对变量`y`赋值：
+ 当在`for`循环内使用同步函数`wait_to_read`时，每次赋值不使用异步计算
+ 当在`for`循环外使用同步函数`waitall`时，则使用异步计算

In [20]:
with Benchmark('synchronous.'):
    for _ in range(1000):
        y = x + 1
        y.wait_to_read()

with Benchmark('asynchronous.'):
    for _ in range(1000):
        y = x + 1
    nd.waitall()

synchronous. time: 7.3877 sec
asynchronous. time: 3.5370 sec


我们观察到，使用异步计算能提升一定的计算性能。为了解释这一现象，让我们对Python前端线程和C++后端线程的交互稍作简化。在每一次循环中，前端和后端的交互大约可以分为3个阶段：
1. 前端令后端将计算任务`y = x + 1`放进队列
2. 后端从队列中获取计算任务并执行真正的计算
3. 后端将计算结果返回给前端

我们将这3个阶段的耗时分别设为$t_1, t_2, t_3$。如果不使用异步计算，执行1000次计算的总耗时大约为$1000 (t_1+ t_2 + t_3)$；如果使用异步计算，由于每次循环中前端都无须等待后端返回计算结果，执行1000次计算的总耗时可以降为$t_1 + 1000 t_2 + t_3$(假设$1000t_2 > 999t_1$)。

### 8.2.4 异步计算对内存的影响
在前面章节中实现的模型训练过程中，我们通常会在每个小批量上评测一下模型，如模型的损失或者准确率。细心的读者也许已经发现了，这类评测常用到同步函数，如`asscalar`函数或者`asnumpy`函数。如果去掉这些同步函数，前端会将大量的小批量计算任务在极短的时间内丢给后端，从而可能导致占用更多内存。当我们在每个小批量上都使用同步函数时，前端在每次迭代时仅会将一个小批量的任务丢给后端执行计算，并通常会减小内存占用。

由于深度学习模型通常比较大，而内存资源通常有限，建议大家在训练模型时对每个小批量都使用同步函数，例如，用`asscalar`函数或者`asnumpy`函数评价模型的表现。类似地，在使用模型预测时，为了减小内存的占用，也建议大家对每个小批量预测时都使用同步函数，例如，直接打印出当前小批量的预测结果。

下面我们来演示异步计算对内存的影响。我们先定义一个数据获取函数`data_iter`，它会从被调用时开始计时，并定期打印到目前为止获取数据批量的总耗时。

In [21]:
def data_iter():
    start = time.time() 
    num_batches, batch_size = 100, 1024 
    for i in range(num_batches):
        X = nd.random.normal(shape=(batch_size, 512)) 
        y = nd.ones((batch_size,)) 
        yield X, y 
        if (i + 1) % 50 == 0:
            print('batch %d, time %f sec' % (i + 1, time.time() - start))

下⾯定义多层感知机、优化算法和损失函数。

In [22]:
net = nn.Sequential()
net.add(
    nn.Dense(2048, activation='relu'), 
    nn.Dense(512, activation='relu'), 
    nn.Dense(1)) 
net.initialize()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.005}) 
loss = gloss.L2Loss()

这⾥定义辅助函数来监测内存的使⽤。需要注意的是，这个函数只能在`Linux`或`macOS`上运⾏。

In [23]:
def get_mem():
    res = subprocess.check_output(['ps', 'u', '-p', str(os.getpid())]) 
    return int(str(res).split()[15]) / 1e3

现在我们可以做测试了。我们先试运⾏⼀次，让系统把`net`的参数初始化。有关初始化的讨论可参⻅`模型参数的延后初始化`⼀节。

In [24]:
for X, y in data_iter():
    break
loss(y, net(X)).wait_to_read()

对于训练模型`net`来说，我们可以自然地使用同步函数`asscalar`将每个小批量的损失从`NDArray`格式中取出，并打印每个迭代周期后的模型损失。此时，每个小批量的生成间隔较长，不过内存开销较小。

In [25]:
l_sum, mem = 0, get_mem()

for X, y in data_iter():
    with autograd.record():
        l = loss(y, net(X)) 
    l_sum += l.mean().asscalar() 
    l.backward() 
    trainer.step(X.shape[0]) 
nd.waitall() 
print('increased memory: %f MB' % (get_mem() - mem))

batch 50, time 13.377381 sec
batch 100, time 26.867962 sec
increased memory: 4.648000 MB


如果去掉同步函数，虽然每个小批量的⽣成间隔较短，但训练过程中可能会导致内存占⽤较⾼。

这是因为在默认异步计算下，前端会将所有小批量计算在短时间内全部丢给后端。这可能在内存积压⼤量中间结果⽆法释放。实验中我们看到，不到⼀秒，所有数据(`X`和`y`)就都已经产⽣。但因为训练速度没有跟上，所以这些数据只能放在内存⾥不能及时清除，从而占⽤额外内存。

In [26]:
mem = get_mem()

for X, y in data_iter():
    with autograd.record():
        l = loss(y, net(X)) 
    l.backward() 
    trainer.step(X.shape[0]) 
nd.waitall() 
print('increased memory: %f MB' % (get_mem() - mem))

batch 50, time 0.083964 sec
batch 100, time 0.166682 sec
increased memory: 0.016000 MB


## 8.3 ⾃动并⾏计算
`MXNet`后端会自动构建计算图。通过计算图，系统可以知道所有计算的依赖关系，并可以选择将没有依赖关系的多个任务并行执行来获得计算性能的提升。例如`异步计算`一节的第一个例子里依次执行了`a = nd.ones((1, 2))`和`b = nd.ones((1, 2))`。这两步计算之间并没有依赖关系，因此系统可以选择并行执行它们。

通常，一个运算符会用到所有CPU或单块GPU上全部的计算资源。例如，`dot`运算符会用到所有CPU或单块GPU上所有的线程。如果每个运算符的计算量足够大，只在CPU上或者单块GPU上并行运行多个运算符时，每个运算符的运行只分到CPU或单块GPU上部分计算资源。即使这些计算可以并行，最终计算性能的提升可能也并不明显。本节中探讨的自动并行计算主要关注同时使用CPU和GPU的并行计算，以及计算和通信的并行。

### 8.3.1 CPU和GPU的并行计算
我们先介绍CPU和GPU的并行计算，例如，程序中的计算既发生在CPU上，又发生在GPU上。先定义`run`函数，令它做10次矩阵乘法。

```python
def run(x):
    return [nd.dot(x, x) for _ in range(10)]
```

接下来，分别在内存和显存上创建`NDArray`。

```python
x_cpu = nd.random.uniform(shape=(2000, 2000))
x_gpu = nd.random.uniform(shape=(6000, 6000), ctx=mx.gpu(0))
```

然后，分别使用它们在CPU和GPU上运行`run`函数并打印运行所需时间。

```python
run(x_cpu)  # 预热开始
run(x_gpu)
nd.waitall()  # 预热结束

with d2l.Benchmark('Run on CPU.'):
    run(x_cpu)
    nd.waitall()

with d2l.Benchmark('Then run on GPU.'):
    run(x_gpu)
    nd.waitall()
```

我们去掉`run(x_cpu)`和`run(x_gpu)`这两个计算任务之间的`waitall`同步函数，并希望系统能自动并行这两个任务。

```python
with d2l.Benchmark('Run on both CPU and GPU in parallel.'):
    run(x_cpu)
    run(x_gpu)
    nd.waitall()
```

可以看到，当两个计算任务一起执行时，执行总时间小于它们分开执行的总和。这表明，MXNet能有效地在CPU和GPU上自动并行计算。


### 8.3.2 计算和通信的并行计算
在同时使用CPU和GPU的计算中，经常需要在内存和显存之间复制数据，造成数据的通信。在下面的例子中，我们在GPU上计算，然后将结果复制回CPU使用的内存。我们分别打印GPU上计算时间和显存到内存的通信时间。

```python
def copy_to_cpu(x):
    return [y.copyto(mx.cpu()) for y in x]

with d2l.Benchmark('Run on GPU.'):
    y = run(x_gpu)
    nd.waitall()

with d2l.Benchmark('Then copy to CPU.'):
    copy_to_cpu(y)
    nd.waitall()
```

我们去掉计算和通信之间的`waitall`同步函数，打印这两个任务完成的总时间。

```python
with d2l.Benchmark('Run and copy in parallel.'):
    y = run(x_gpu)
    copy_to_cpu(y)
    nd.waitall()
```

可以看到，执行计算和通信的总时间小于两者分别执行的耗时之和。需要注意的是，这个计算并通信的任务不同于本节之前介绍的同时使用CPU和GPU并行计算的任务。这里的运行和通信之间有依赖关系：`y[i]`必须先在GPU上计算好才能复制到CPU使用的内存。所幸的是，在计算`y[i]`的时候系统可以复制`y[i-1]`，从而减少计算和通信的总运行时间。

## 8.4 多GPU计算
本节中我们将展⽰如何使⽤多块GPU计算，例如，使⽤多块GPU训练同⼀个模型。正如所期望的那样，运⾏本节中的程序需要⾄少2块GPU。事实上，⼀台机器上安装多块GPU很常⻅，这是因为主板上通常会有多个PCIe插槽。如果正确安装了NVIDIA驱动，我们可以通过`nvidia-smi`命令来查看当前计算机上的全部GPU。

<img src="images/08_00.png" style="width:600px;"/>

`自动并行计算`一节介绍过，大部分运算可以使用所有的CPU的全部计算资源，或者单块GPU的全部计算资源。但如果使用多块GPU训练模型，我们仍然需要实现相应的算法。这些算法中最常用的叫作数据并行。


### 8.4.1 数据并行
数据并行目前是深度学习里使用最广泛的将模型训练任务划分到多块GPU的方法。下面我们就以小批量随机梯度下降为例来介绍数据并行是如何工作的。

假设一台机器上有$k$块GPU。给定需要训练的模型，每块GPU及其相应的显存将分别独立维护一份完整的模型参数。在模型训练的任意一次迭代中，给定一个随机小批量，我们将该批量中的样本划分成$k$份并分给每块显卡的显存一份。然后，每块GPU将根据相应显存所分到的小批量子集和所维护的模型参数分别计算模型参数的本地梯度。接下来，我们把$k$块显卡的显存上的本地梯度相加，便得到当前的小批量随机梯度。之后，每块GPU都使用这个小批量随机梯度分别更新相应显存所维护的那一份完整的模型参数。

`图8.1`描绘了使用2块GPU的数据并行下的小批量随机梯度的计算。

<img src="images/08_01.png" style="width:600px;"/>

### 8.4.2 定义模型
我们使用`卷积神经网络(LeNet)`一节里介绍的LeNet来作为本节的样例模型。这里的模型实现部分只用到了`NDArray`。

```python
# 初始化模型参数
scale = 0.01
W1 = nd.random.normal(scale=scale, shape=(20, 1, 3, 3))
b1 = nd.zeros(shape=20)
W2 = nd.random.normal(scale=scale, shape=(50, 20, 5, 5))
b2 = nd.zeros(shape=50)
W3 = nd.random.normal(scale=scale, shape=(800, 128))
b3 = nd.zeros(shape=128)
W4 = nd.random.normal(scale=scale, shape=(128, 10))
b4 = nd.zeros(shape=10)
params = [W1, b1, W2, b2, W3, b3, W4, b4]

# 定义模型
def lenet(X, params):
    h1_conv = nd.Convolution(data=X, weight=params[0], bias=params[1], kernel=(3, 3), num_filter=20)
    h1_activation = nd.relu(h1_conv)
    h1 = nd.Pooling(data=h1_activation, pool_type='avg', kernel=(2, 2), stride=(2, 2))
    h2_conv = nd.Convolution(data=h1, weight=params[2], bias=params[3], kernel=(5, 5), num_filter=50)
    h2_activation = nd.relu(h2_conv)
    h2 = nd.Pooling(data=h2_activation, pool_type='avg', kernel=(2, 2), stride=(2, 2))
    h2 = nd.flatten(h2)
    h3_linear = nd.dot(h2, params[4]) + params[5]
    h3 = nd.relu(h3_linear)
    y_hat = nd.dot(h3, params[6]) + params[7]
    return y_hat

# 交叉熵损失函数
loss = gloss.SoftmaxCrossEntropyLoss()
```

### 8.4.3 多GPU之间同步数据
我们需要实现一些多GPU之间同步数据的辅助函数。下面的`get_params`函数将模型参数复制到某块显卡的显存并初始化梯度。

```python
def get_params(params, ctx):
    new_params = [p.copyto(ctx) for p in params]
    for p in new_params:
        p.attach_grad()
    return new_params
```

尝试把模型参数`params`复制到`gpu(0)`上。

```python
new_params = get_params(params, mx.gpu(0))
print('b1 weight:', new_params[1])
print('b1 grad:', new_params[1].grad)
```

输出：
```bash
b1 weight:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
<NDArray 20 @gpu(0)> 

b1 grad:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
<NDArray 20 @gpu(0)>
```

给定分布在多块显卡的显存之间的数据。下面的`allreduce`函数可以把各块显卡的显存上的数据加起来，然后再广播到所有的显存上。

```python
def allreduce(data):
    for i in range(1, len(data)):
        data[0][:] += data[i].copyto(data[0].context)
    for i in range(1, len(data)):
        data[0].copyto(data[i])
```

简单测试一下`allreduce`函数。

```python
data = [nd.ones((1, 2), ctx=mx.gpu(i)) * (i + 1) for i in range(2)]
print('before allreduce:', data)
allreduce(data)
print('after allreduce:', data)
```

输出：
```bash
before allreduce: [ 
[[1. 1.]] 
<NDArray 1x2 @gpu(0)>,
[[2. 2.]] 
<NDArray 1x2 @gpu(1)>] after allreduce: [ 
[[3. 3.]] 
<NDArray 1x2 @gpu(0)>, 
[[3. 3.]] 
<NDArray 1x2 @gpu(1)>]
```

给定一个批量的数据样本，下面的`split_and_load`函数可以将其划分并复制到各块显卡的显存上。

```python
def split_and_load(data, ctx):
    n, k = data.shape[0], len(ctx)
    m = n // k  # 简单起见，假设可以整除
    assert m * k == n, '# examples is not divided by # devices.'
    return [data[i * m: (i + 1) * m].as_in_context(ctx[i]) for i in range(k)]
```

让我们试着用`split_and_load`函数将6个数据样本平均分给2块显卡的显存。

```python
batch = nd.arange(24).reshape((6, 4))
ctx = [mx.gpu(0), mx.gpu(1)]
splitted = split_and_load(batch, ctx)
print('input: ', batch)
print('load into', ctx)
print('output:', splitted)
```

输出：
```bash
input:[
[ 0. 1. 2. 3.] 
[ 4. 5. 6. 7.] 
[ 8. 9. 10. 11.] 
[12. 13. 14. 15.]
[16. 17. 18. 19.] 
[20. 21. 22. 23.]] 
<NDArray 6x4 @cpu(0)> 
load into [gpu(0), gpu(1)] 
output: [ 
[[ 0. 1. 2. 3.] 
[ 4. 5. 6. 7.] 
[ 8. 9. 10. 11.]] 
<NDArray 3x4 @gpu(0)>, 
[[12. 13. 14. 15.]
[16. 17. 18. 19.]
[20. 21. 22. 23.]] 
<NDArray 3x4 @gpu(1)>
]
```

### 8.4.4 单个⼩批量上的多GPU训练
现在我们可以实现单个小批量上的多GPU训练了。它的实现主要依据本节介绍的数据并行方法。我们将使用刚刚定义的多GPU之间同步数据的辅助函数`allreduce`和`split_and_load`。

```python
def train_batch(X, y, gpu_params, ctx, lr):
    # 当ctx包含多块GPU及相应的显存时，将小批量数据样本划分并复制到各个显存上
    gpu_Xs, gpu_ys = split_and_load(X, ctx), split_and_load(y, ctx)
    with autograd.record():  # 在各块GPU上分别计算损失
        ls = [loss(lenet(gpu_X, gpu_W), gpu_y)
              for gpu_X, gpu_y, gpu_W in zip(gpu_Xs, gpu_ys, gpu_params)]
    for l in ls:  # 在各块GPU上分别反向传播
        l.backward()
    # 把各块显卡的显存上的梯度加起来，然后广播到所有显存上
    for i in range(len(gpu_params[0])):
        allreduce([gpu_params[c][i].grad for c in range(len(ctx))])
    for param in gpu_params:  # 在各块显卡的显存上分别更新模型参数
        d2l.sgd(param, lr, X.shape[0])  # 这里使用了完整批量大小
```

### 8.4.5 定义训练函数
现在我们可以定义训练函数了。这里的训练函数和`softmax回归的从零开始实现`一节定义的训练函数`train_ch3`有所不同。值得强调的是，在这里我们需要依据数据并行将完整的模型参数复制到多块显卡的显存上，并在每次迭代时对单个小批量进行多GPU训练。

```python
def train(num_gpus, batch_size, lr):
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    ctx = [mx.gpu(i) for i in range(num_gpus)]
    print('running on:', ctx)
    # 将模型参数复制到num_gpus块显卡的显存上
    gpu_params = [get_params(params, c) for c in ctx]
    for epoch in range(4):
        start = time.time()
        for X, y in train_iter:
            # 对单个小批量进行多GPU训练
            train_batch(X, y, gpu_params, ctx, lr)
            nd.waitall()
        train_time = time.time() - start

        def net(x):  # 在gpu(0)上验证模型
            return lenet(x, gpu_params[0])

        test_acc = d2l.evaluate_accuracy(test_iter, net, ctx[0])
        print('epoch %d, time %.1f sec, test acc %.2f'
              % (epoch + 1, train_time, test_acc))
```

### 8.4.6 多GPU训练实验
让我们先从单GPU训练开始。设批量大小为256，学习率为0.2。

```python
train(num_gpus=1, batch_size=256, lr=0.2)
```

输出：
```bash
running on: [gpu(0)] 
epoch 1, time 1.5 sec, test acc 0.10 
epoch 2, time 1.4 sec, test acc 0.60 
epoch 3, time 1.4 sec, test acc 0.75 
epoch 4, time 1.4 sec, test acc 0.68
```

保持批量大小和学习率不变，将使用的GPU数量改为2。可以看到，测试准确率的提升同上一个实验中的结果大体相当。因为有额外的通信开销，所以我们并没有看到训练时间的显著降低。因此，我们将在下一节实验计算更加复杂的模型。

```python
train(num_gpus=2, batch_size=256, lr=0.2)
```

输出：
```bash
running on: [gpu(0), gpu(1)] 
epoch 1, time 2.6 sec, test acc 0.10
epoch 2, time 2.6 sec, test acc 0.69 
epoch 3, time 2.4 sec, test acc 0.74 
epoch 4, time 2.5 sec, test acc 0.74
```

## 8.5 多GPU计算的简洁实现
在Gluon中，我们可以很方便地使用数据并行进行多GPU计算。我们并不需要自己实现`多GPU计算`一节里介绍的多GPU之间同步数据的辅助函数。

运行本节中的程序需要至少2块GPU。

### 8.5.1 多GPU上初始化模型参数
我们使用ResNet-18作为本节的样例模型。由于本节的输入图像使用原尺寸，这里的模型构造与`残差网络(ResNet)`一节中的`ResNet-18`构造稍有不同。这里的模型在一开始使用了较小的卷积核、步幅和填充，并去掉了最大池化层。

```python
def resnet18(num_classes):  # 本函数已保存在d2lzh包中方便以后使用
    def resnet_block(num_channels, num_residuals, first_block=False):
        blk = nn.Sequential()
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.add(d2l.Residual(
                    num_channels, use_1x1conv=True, strides=2))
            else:
                blk.add(d2l.Residual(num_channels))
        return blk

    net = nn.Sequential()
    # 这里使用了较小的卷积核、步幅和填充，并去掉了最大池化层
    net.add(nn.Conv2D(64, kernel_size=3, strides=1, padding=1),
            nn.BatchNorm(), nn.Activation('relu'))
    net.add(resnet_block(64, 2, first_block=True),
            resnet_block(128, 2),
            resnet_block(256, 2),
            resnet_block(512, 2))
    net.add(nn.GlobalAvgPool2D(), nn.Dense(num_classes))
    return net

net = resnet18(10)
```

之前我们介绍了如何使用`initialize`函数的`ctx`参数在内存或单块显卡的显存上初始化模型参数。事实上，`ctx`可以接受一系列的CPU及内存和GPU及相应的显存，从而使初始化好的模型参数复制到`ctx`里所有的内存和显存上。

```python
ctx = [mx.gpu(0), mx.gpu(1)]
net.initialize(init=init.Normal(sigma=0.01), ctx=ctx)
```

`Gluon`提供了上一节中实现的`split_and_load`函数。它可以划分一个小批量的数据样本并复制到各个内存或显存上。之后，根据输入数据所在的内存或显存，模型计算会相应地使用CPU或相同显卡上的GPU。

```python
x = nd.random.uniform(shape=(4, 1, 28, 28))
gpu_x = gutils.split_and_load(x, ctx)
net(gpu_x[0]), net(gpu_x[1])
```

输出：
```bash
Out[4]: (
[[ 5.48149410e-06 -8.33710715e-07 -1.63167692e-06 -6.36740651e-07
-3.82161625e-06 -2.35140487e-06 -2.54695942e-06 -9.47847525e-08
-6.90336265e-07 2.57562351e-06] 
[ 5.47108630e-06 -9.42464624e-07 -1.04940636e-06 9.80811592e-08
-3.32518175e-06 -2.48629181e-06 -3.36428002e-06 1.04558694e-07
-6.10013558e-07 2.03278455e-06]] 
<NDArray 2x10 @gpu(0)>, 
[[ 5.61763409e-06 -1.28375871e-06 -1.46055413e-06 1.83029556e-07
-3.55116504e-06 -2.43710201e-06 -3.57318004e-06 -3.09748373e-07
-1.10165661e-06 1.89098932e-06]
[ 5.14186922e-06 -1.37299264e-06 -1.15200896e-06 1.15074045e-07
-3.73728130e-06 -2.82897167e-06 -3.64771950e-06 1.57815748e-07
-6.07329866e-07 1.97120107e-06]] 
<NDArray 2x10 @gpu(1)>)
```

现在，我们可以访问已初始化好的模型参数值了。需要注意的是，默认情况下`weight.data()`会返回内存上的参数值。因为我们指定了2块GPU来初始化模型参数，所以需要指定显存来访问参数值。我们看到，相同参数在不同显卡的显存上的值一样。

```python
weight = net[0].params.get('weight')

try:
    weight.data()
except RuntimeError:
    print('not initialized on', mx.cpu())
weight.data(ctx[0])[0], weight.data(ctx[1])[0]
```

输出：
```bash
not initialized on cpu(0)

Out[5]: (
[[[-0.01473444 -0.01073093 -0.01042483] 
[-0.01327885 -0.01474966 -0.00524142] 
[ 0.01266256 0.00895064 -0.00601594]]] 
<NDArray 1x3x3 @gpu(0)>, 

[[[-0.01473444 -0.01073093 -0.01042483]
[-0.01327885 -0.01474966 -0.00524142]
[ 0.01266256 0.00895064 -0.00601594]]] 
<NDArray 1x3x3 @gpu(1)>)
```

### 8.5.2 多GPU训练模型
当使用多块GPU来训练模型时，`Trainer`实例会自动做数据并行，例如，划分小批量数据样本并复制到各块显卡的显存上，以及对各块显卡的显存上的梯度求和再广播到所有显存上。这样，我们就可以很方便地实现训练函数了。

```python
def train(num_gpus, batch_size, lr):
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    ctx = [mx.gpu(i) for i in range(num_gpus)]
    print('running on:', ctx)
    net.initialize(init=init.Normal(sigma=0.01), ctx=ctx, force_reinit=True)
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr})
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(4):
        start = time.time()
        for X, y in train_iter:
            gpu_Xs = gutils.split_and_load(X, ctx)
            gpu_ys = gutils.split_and_load(y, ctx)
            with autograd.record():
                ls = [loss(net(gpu_X), gpu_y)
                      for gpu_X, gpu_y in zip(gpu_Xs, gpu_ys)]
            for l in ls:
                l.backward()
            trainer.step(batch_size)
        nd.waitall()
        train_time = time.time() - start
        test_acc = d2l.evaluate_accuracy(test_iter, net, ctx[0])
        print('epoch %d, time %.1f sec, test acc %.2f' % (
            epoch + 1, train_time, test_acc))
```

首先在单块GPU上训练模型。

```python
train(num_gpus=1, batch_size=256, lr=0.1)
```

输出：
```bash
running on: [gpu(0)] 
epoch 1, time 14.0 sec, test acc 0.89 
epoch 2, time 12.9 sec, test acc 0.91 
epoch 3, time 13.0 sec, test acc 0.91 
epoch 4, time 13.0 sec, test acc 0.92
```

然后尝试在2块GPU上训练模型。与上一节使用的`LeNet`相比，`ResNet-18`的计算更加复杂，通信时间比计算时间更短，因此`ResNet-18`的并行计算所获得的性能提升更佳。

```python
train(num_gpus=2, batch_size=512, lr=0.2)
```

输出：
```bash
running on: [gpu(0), gpu(1)]
epoch 1, time 7.3 sec, test acc 0.69 
epoch 2, time 6.7 sec, test acc 0.86 
epoch 3, time 6.7 sec, test acc 0.89
epoch 4, time 6.7 sec, test acc 0.90
```